In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import linregress

In [3]:
!pip install openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 2.1 MB/s eta 0:00:0000:0100:01


In [4]:
# Chargement de toutes les données
file_path = "data/database-imputed ESG.xlsx"
all_data = pd.read_excel(file_path)

In [8]:
# Variables disponibles
variables = ["market_cap", "employees", "revenue", "scope_1", "scope_2", "scope_3",
                         "waste_production", "waste_recycling", "water_consumption",
                         "water_withdrawal", "energy_consumption", "hours_of_training",
                         "gender_pay_gap", "independent_board_members_percentage",
                         "legal_costs_paid_for_controversies", "ceo_compensation"]

variables_env = ["scope_1", "scope_2", "scope_3",
                "waste_production", "waste_recycling", "water_consumption",
                "water_withdrawal", "energy_consumption"]

variables_eg = ["hours_of_training","gender_pay_gap", "independent_board_members_percentage",
                "legal_costs_paid_for_controversies", "ceo_compensation"]

# Années 
annees = ["2018", "2019", "2020"]

# Regions 
regions = ["Africa / Middle East", "Asia / Pacific", "Europe", 
           "Latin America and Caribbean", "United States and Canada"]

# Secteurs 
secteurs = all_data['primary_industry'].unique()

In [12]:
max = 0
max_var = ["", "", "", "", ""]

#for x in variables_env, y in variables_eg, a in annees, r in regions, s in secteurs :
for x, y, a, r, s in zip(variables_env, variables_eg, annees, regions, secteurs) :
    data = pd.DataFrame(all_data[(all_data["year"] == int(a)) & (all_data["primary_industry"] == s) & (all_data["region"] == r)])
    model = np.polyfit(data[x], data[y], 1)
    slope, intercept, r_value, p_value, std_err = linregress(data[x], data[y])
    r_squared = r_value ** 2
    if r_squared > max :
        max_var = [x, y, a, r, s]
        max = r_squared

print(max_var, max)

['scope_2', 'gender_pay_gap', '2019', 'Asia / Pacific', 'Integrated Oil and Gas'] 0.8293546708823886


/tmp/ipykernel_261/3238545959.py:7: RankWarning: Polyfit may be poorly conditioned
  model = np.polyfit(data[x], data[y], 1)
/opt/mamba/lib/python3.10/site-packages/scipy/stats/_stats_mstats_common.py:182: RuntimeWarning: invalid value encountered in scalar divide
  slope = ssxym / ssxm
/opt/mamba/lib/python3.10/site-packages/scipy/stats/_stats_mstats_common.py:196: RuntimeWarning: invalid value encountered in sqrt
  t = r * np.sqrt(df / ((1.0 - r + TINY)*(1.0 + r + TINY)))
/opt/mamba/lib/python3.10/site-packages/scipy/stats/_stats_mstats_common.py:199: RuntimeWarning: invalid value encountered in scalar divide
  slope_stderr = np.sqrt((1 - r**2) * ssym / ssxm / df)
